In [1]:
!pip install streamlit

  Using cached streamlit-0.75.0-py2.py3-none-any.whl (7.5 MB)
Processing c:\users\pichau\appdata\local\pip\cache\wheels\b7\a5\68\fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4\blinker-1.4-py3-none-any.whl
  Using cached validators-0.18.2-py3-none-any.whl (19 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached watchdog-1.0.2-py3-none-win_amd64.whl (72 kB)
  Using cached GitPython-3.1.12-py3-none-any.whl (159 kB)
  Using cached base58-2.1.0-py3-none-any.whl (5.6 kB)
  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached gitdb-4.0.5-py3-none-any.whl (63 kB)
  Using cached smmap-3.0.5-py2.py3-none-any.whl (25 kB)


In [13]:
#import streamlit as st
df = pd.read_csv('data/wine.data', sep = ',',names=range(1, 15, 1))
x = df.drop(columns=[1]).copy()
y = df[1].copy()
display(df)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [128]:
!pip install bokeh

  Created wheel for bokeh: filename=bokeh-2.2.3-py3-none-any.whl size=9296319 sha256=51d591d950cffed57a8591d0070a8a6c3f6065378924388febab2a18cf535371
  Stored in directory: c:\users\pichau\appdata\local\pip\cache\wheels\44\48\ac\5cde9e6d0b1e0afb4cff564eaf59462468abe37cfc83cbb5ed
Successfully built bokeh


In [180]:
%%writefile init.py
import streamlit as st
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from bokeh.plotting import figure
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

df = pd.read_csv('data/wine.data', sep = ',',names=range(1, 15, 1))

st.title('Principal Component Analysis (PCA)')
st.markdown("""
Esse é um aplicativo simples para analisarmos as componentes principais de um dataset de Classificação de Vinhos. Os dados são resultados de uma análise química de vinhos cultivados na mesma região da Itália, mas derivados de três cultivares diferentes. A análise determinou as quantidades de 13 constituintes encontrados em cada um dos três tipos de vinhos.
* **Python libraries:** pandas, numpy, scikit-learn and streamlit 
* **Data source:** [Dataset-Wine](https://archive.ics.uci.edu/ml/datasets/Wine)
""")

st.sidebar.header('Parâmetros do Modelo')

def user_input_features():
    num_componentes = st.sidebar.slider('Número de Componentes Principais: ',1,13,5)
    num_vizinhos = st.sidebar.slider('Número de Vizinhos mais próximos(KNN): ',1,10,5)
    return num_componentes, num_vizinhos

#num_comp = user_input_features()
    
st.header('Exibição do Dataset')
st.write("Dimensão dos dados {} linhas e {} colunas".format(str(df.shape[0]), str(df.shape[1])))
st.dataframe(df)

def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download = "wine.csv">Dowload CSV File</a>'
    return href

st.markdown(filedownload(df), unsafe_allow_html=True)

#Mapa de calor
if st.button('Mapa de Calor'):
    st.header('Mapa de calor de correlação dos dados')
    st.write('Por esse mapa de calor a gente ver com as variáveis se relacionam entre si.')
    corr = df.drop(columns=[1]).corr()
    mask = np.zeros_like(corr)
    mask [np.triu_indices_from(mask)] = True
    with sns.axes_style('white'):
        f,ax=plt.subplots(figsize=(7,5))
        ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
    st.pyplot(f)

#Dados Padronizados
scaled = df.to_numpy()
x = scaled[:, 1:14].copy()
y = scaled[:, 0].copy()
x = StandardScaler().fit_transform(x)
pca = PCA()
principalComponents = pca.fit_transform(x)
names = []
for i in range(1, 14, 1):
    names.append('PC {}'.format(i))
names.append('Target')
df_autovetores=pd.DataFrame(principalComponents)
finalDf = pd.concat([df_autovetores, df [[1]]], axis = 1)
finalDf.columns = names

#Autovalores e autovetore
if st.button('Principais Componentes'):
    st.header('Dataset em função das suas componentes principais')
    st.write('Quando padronizamos os nossos dados, nós fazemos duas operações. Centralizamos o espaço de dados, para evidenciar as relações lineares que existem entre os atributos. E, fazemos o reescalonamento dos dados, para as medidas de escalas não atrapalharem na hora da produção do nosso modelo.')
    st.dataframe(finalDf)

#Variância explicada
if st.button('Variância explicada'):
    st.header('Soma acumulada das variâncias')
    st.write('Toda componente principal tem um autovalor associado, as componentes de mais importância tem os maiores autovalores. A soma dos autovalores é igual soma acumulada da variância dos nossos dados originais. Desse modo, podemos diminuir a dimensão do nosso dataset sem perder muita informação ao definir de quanto de informação queremos. Quando diminuimos a dimensão do nosso dataset, pode facilitar na análise dos nossos dados e diminuir redudâncias de dados.')
    matrizcov = np.corrcoef(x, rowvar=0)
    autovalores, _ = np.linalg.eig(matrizcov)
    idx = autovalores.argsort()[::-1]   
    autovalores = autovalores[idx]
    soma_acumulada = []
    soma_auto = np.sum(autovalores)
    soma = 0
    for i in range(0, len(autovalores)):
        if len(soma_acumulada) == 0:
            soma+=(autovalores[i]/soma_auto)*100
            soma_acumulada.append(soma)
        else:
            soma+=(autovalores[i]/soma_auto)*100
            soma_acumulada.append(soma)
    df_var_explicada= pd.DataFrame(soma_acumulada, columns=['VariânciaExplicada'])
    df_var_explicada.index = range(1,14,1)
    p = figure(
        title='Soma acumulada das importâncias de cada componente principal:',
        x_axis_label='Número de Componentes',
        y_axis_label='Variância Explicada(%)')
    
    
    p.scatter(df_var_explicada.index, df_var_explicada['VariânciaExplicada'], line_width=2)
    #st.pyplot(fig)
    st.bokeh_chart(p, use_container_width=True)

X = finalDf.drop(columns=['Target']).copy()
Y = finalDf['Target'].copy()
#Número de componentes principais que eu vou usar
num_pcs, num_vizinho = user_input_features()

x_train, x_test, y_train, y_test = train_test_split(X[:][names[:num_pcs]], Y, test_size = 0.30)
knn = KNeighborsClassifier(n_neighbors=num_vizinho)
#Train the model using the training sets
knn.fit(x_train, y_train)
#Predict the response for test dataset
y_pred = knn.predict(x_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct
st.header('Criando um modelo usando PCA')
st.write("""
Para fins didáticos você pode passar alguns parâmetros para criarmos um modelo de machine learning. Você pode passar o número de componentes principais, a fim de visualizar como o número de dimensões pode afetar a criação do nosso modelo. Além disso, vamos usar o algoritmo K-Nearest Neighbors(KNN) para criarmos o nosso modelo, assim, pode definir o número de vizinhos mais próximos que o algoritmo tem que usar para criar um modelo para as componentes principais e o dataset original. Assim, vamos comparar as acurácias entre as componentes principais e o dataset original. 
""")
st.subheader('Performance do dataset usando {} Componentes Principais'.format(num_pcs))
st.write("Acurácia: ",metrics.accuracy_score(y_test, y_pred))

#dataset original
X2 = df.drop(columns=[1]).copy()
Y2 = df[1].copy()
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size = 0.30)
knn2 = KNeighborsClassifier(n_neighbors=num_vizinho)
#Train the model using the training sets
knn2.fit(x_train2, y_train2)
#Predict the response for test dataset
y_pred2 = knn2.predict(x_test2)
st.subheader('Performance do dataset original usando {} vizinhos'.format(num_vizinho))
st.write("Acurácia: ",metrics.accuracy_score(y_test2, y_pred2))

Overwriting teste.py


In [148]:
names[:5]

['PC 1', 'PC 2', 'PC 3', 'PC 4', 'PC 5']

In [71]:
from sklearn.decomposition import PCA
names = []
for i in range(1, 14, 1):
    names.append('PC {}'.format(i))

pca = PCA()
principalComponents = pca.fit_transform(x)

In [52]:
df_autovetores=pd.DataFrame(principalComponents)
# Change the column names 
df_autovetores.columns = names

In [72]:
print(pca.explained_variance_ratio_)

[9.98091230e-01 1.73591562e-03 9.49589576e-05 5.02173562e-05
 1.23636847e-05 8.46213034e-06 2.80681456e-06 1.52308053e-06
 1.12783044e-06 7.21415811e-07 3.78060267e-07 2.12013755e-07
 8.25392788e-08]


In [75]:
matrizcov = np.corrcoef(x, rowvar=0)
autovalores, _ = np.linalg.eig(matrizcov)
idx = autovalores.argsort()[::-1]   
autovalores = autovalores[idx]
num_componentes = []
for i in range(1, len(autovalores)+1):
    num_componentes.append(i)
df_var_explicada = pd.DataFrame()
df_var_explicada['num_componentes'] = num_componentes
df_var_explicada['autovalor'] = (autovalores/np.sum(autovalores))*100
display(df_var_explicada)
#autovetores = autovetores[:,idx]

In [76]:
autovalores

array([4.70585025, 2.49697373, 1.44607197, 0.91897392, 0.85322818,
       0.64165703, 0.55102831, 0.34849736, 0.28887994, 0.25090248,
       0.22578864, 0.16877023, 0.10337794])

In [85]:
num_componentes = []
for i in range(1, len(autovalores)+1):
    num_componentes.append(i)
df_var_explicada = pd.DataFrame()
df_var_explicada['num_componentes'] = num_componentes
df_var_explicada['autovalor'] = (autovalores/np.sum(autovalores))*100
display(df_var_explicada)

,num_componentes,autovalor
0,1,36.198848
1,2,19.207490
2,3,11.123631
3,4,7.069030
4,5,6.563294
5,6,4.935823
6,7,4.238679
7,8,2.680749
8,9,2.222153
9,10,1.930019


In [110]:
#st.line_chart(df)
soma_acumulada = []
soma_auto = np.sum(autovalores)
soma = 0
for i in range(0, len(autovalores)):
    if len(soma_acumulada) == 0:
        soma+=(autovalores[i]/soma_auto)*100
        soma_acumulada.append(soma)
    else:
        soma+=(autovalores[i]/soma_auto)*100
        soma_acumulada.append(soma)
df_var_explicada= pd.DataFrame(soma_acumulada, columns=['VariânciaExplicada'])
df_var_explicada.index = range(1,14,1)
df_var_explicada

,VariânciaExplicada
1,36.198848
2,55.406338
3,66.529969
4,73.598999
5,80.162293
6,85.098116
7,89.336795
8,92.017544
9,94.239698
10,96.169717
